# Accessing NeuroHub using the CBRAIN API

## 0. Set up boilerplate

You **must** make sure the module `httpproxy/1.0` is loaded in your environment before starting this notebook on [Béluga JupyterHub](https://jupyterhub.beluga.computecanada.ca/). **Do not** load the module in your `.bashrc`, it interferes with initial login, this is needed once you have a JupyterLab instance up and running.

Let's first import libraries that are required to send HTTP requests and manipulate JSON data.

In [ ]:
import requests
import json
import getpass

We define the API endpoint. See the [CBRAIN API](https://portal.cbrain.mcgill.ca/swagger#/) for more information about how to use the API.

In [ ]:
base_url = 'https://portal.cbrain.mcgill.ca'

## 1. Connect to NeuroHub

Connect to NeuroHub with a username and a password or a generated API token.  See the [NeuroHub Portal](https://neurohub.ca/portal.html) for information about how to register for an account.

### 1.b Connect with username/password

We can use username/password authentication where we `POST` our `credentials` to `/session` and get a token that will need to be sent as a parameter within each subsequent request.

In [ ]:
credentials = {
    'login': input('username'),
    'password': getpass.getpass('password')
}

response = requests.post(
    url = '/'.join([base_url, 'session']),
    data = credentials,
    headers = {'Accept': 'application/json'}
)

if response.status_code != requests.codes.ok:
    print('Login failed.')
    print(response)
else:
    print('Login successful.')
    session_info = response.json()

    cbrain_api_token = session_info['cbrain_api_token']
    user_id = str(session_info['user_id'])
    print("User ID:", user_id)

### 1.b Connect with an API token

Alternatively, we can generate a [NeuroHub API Token](https://portal.neurohub.ca/myaccount) via the NeuroHub Portal itself.  Copy the generated new API token and paste it below then we'll get your user ID.

In [ ]:
cbrain_api_token = "PASTE_API_TOKEN_HERE"

response = requests.post(
    url = '/'.join([base_url, 'session']),
    data = {'cbrain_api_token': cbrain_api_token},
    headers = {'Accept': 'application/json'}
)

if response.status_code != requests.codes.ok:
    print('Session not valid.')
    print(response)
else:
    print('Session valid.')
    session_info = response.json()

    user_id = str(session_info['user_id'])
    print("User :", user_id)

## 2. User files

### 2.a Get all user-accessible files 

***WARNING:*** This step may take many minutes. You may want to skip this and the following subsection.

We `GET` our `request` from `/userfiles` for a list of all the files we have access to.  Notice that we page through the results, requesting a subsequent page until done.

In [ ]:
try:

    user_id = str(session_info['user_id'])
    cbrain_api_token = session_info['cbrain_api_token']

except NameError:
    raise UserWarning('Login first.')
 
files = []

request = {
    'cbrain_api_token': cbrain_api_token,
    'page': 1,
    'per_page': 1000
}

while True:

    response = requests.get(
        url = '/'.join([base_url, 'userfiles']),
        data = request,
        headers = {'Accept': 'application/json'}
    )

    if response.status_code != requests.codes.ok:
        print('User files request failed.')
        print(response)
        break

    # Collect the responses on this page then increment
    files += response.json()
    request['page'] += 1
    
    # Stop requesting responses when we're at the last page
    if len(response.json()) < request['per_page']:
        break 
    
print("{} files found\n".format(str(len(files))))

### 2.b Filter files
The list of files above can be filtered on any of their properties.  Here we filter for a name that contains the `.nii.gz` extension.

In [ ]:
results = list(filter(lambda f: '.nii.gz' in f['name'], files))
print("{} results".format(str(len(results))))

Let's arbitrarily get the first of these files.

In [ ]:
file_info = results[0]
file_id = str(file_info['id'])
print(json.dumps(file_info, indent=4))

### 2.c Get a single file

Alternatively, we can get the file directly if we know it's file ID. You can get the file ID by navigating the the File Info page on the NeuroHub portal and note the digits in the URL: https://portal.neurohub.ca/userfiles/FILE_ID

In [ ]:
file_id = "PASTE_FILE_ID_HERE"
file_info = requests.get(
    url = '/'.join([base_url, 'userfiles', file_id]),
    data = {'cbrain_api_token': cbrain_api_token},
    headers = {'Accept': 'application/json'}
)
print(json.dumps(file_info.json(), indent=4))

### 2.d Get file contents

We `GET` the user file content from `/userfiles/{id}/content` for the first filtered result.

In [ ]:
response = requests.get(
    url = '/'.join([base_url, 'userfiles', file_id, 'content']),
    data = {'cbrain_api_token': cbrain_api_token},
    headers = {'Accept': 'application/json'}
)
print(response.headers)

file_content = response.content

### 2.e Process data

You can now do any interactive processing with `response.content`.

Let's compare the computed size to the stated size.

In [ ]:
print(len(file_content))
print(file_info['size'])

## 3. Tasks and Pipelines
### 3.a List possible pipelines

The Tool Configuration ID allows us to specify the pipeline we want to use and on which Bourreaux we want to run it. By browsing all such pipelines, we can note the Tool Configuration ID for a tool-Bourreaux combination.

In [ ]:
try:

    user_id = str(session_info['user_id'])
    cbrain_api_token = session_info['cbrain_api_token']

except NameError:
    raise UserWarning('Login first.')
 
tools = []

request = {
    'cbrain_api_token': cbrain_api_token,
    'page': 1,
    'per_page': 1000
}

while True:

    response = requests.get(
        url = '/'.join([base_url, 'tool_configs']),
        data = request,
        headers = {'Accept': 'application/json'}
    )

    if response.status_code != requests.codes.ok:
        print('Tools request failed.')
        print(response)
        break

    # Collect the responses on this page then increment
    tools += response.json()
    request['page'] += 1
    
    # Stop requesting responses when we're at the last page
    if len(response.json()) < request['per_page']:
        break 
    
print("{} tool configs found\n".format(str(len(tools))))

As we did before for the files, we can now filter the tools by words in their description. In the following cell we filter to find all tools with the word 'FSL' in the description. The tool config ID is the 'id' field. You can use the 'bourreau_id' and 'tool_id' to verify this is the correct tool you wish to use.

In [ ]:
result = list(filter(lambda f: 'FSL' in f['description'], tools))
print(json.dumps(result, indent=4, sort_keys=True))

### 3.b Submit a task

For this example we will submit a task using a .nii.gz file to FSLFirst (tool_id: 51) on Converter-2 (bourreau_id: 39). This task has a Tool Config ID of 721, which we will use to post a task to CBRAIN.

**Note that this needs a valid userfile ID to run. A public data provider to hold example files for this notebook is in the works.**

In [ ]:
#Store the ID of the file we wish to run a task on. In this case, the first filtered result from the
#files loaded in the previous section. Feel free to change it to a brainscan you have access to.
fileID = result[0]['id']

#Write the post API call
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
}
params = (
    ('cbrain_api_token', cbrain_api_token),
)

#Specify the parameters of the task in a dictionary. We are running FSLFirst to register the brain scan.
data = {
  "cbrain_task": {
    'type': 'CbrainTask::FslFirst', 
    'user_id': user_id,
    'tool_config_id': 721, #This specifies the Tool we are running, and the server we are running it on.
    'params': { #These parameters vary from tool to tool.
      'interface_userfile_ids': [fileID], 
      'input_file': fileID, 
      'prefix': 'output', 
      'brain_extracted': False, 
      'three_stage': False, 
      'verbose': False       
    }, 
    'run_number': None, 
    'results_data_provider_id': 179, 
    'cluster_workdir_size': None, 
    'workdir_archived': True, 
    'description': ''}
}

#Convert our parameter dictionary to a json and post the task.
y = json.dumps(data)
response = requests.post('https://portal.cbrain.mcgill.ca/tasks', headers=headers, params=params, data=y)

if response.status_code == 200:
    print(response.text)
    json_format = json.loads(response.text) #save the response to a json file for future use
else:
    print("Task posting failed.")
    print(response.text)

### 3.c Get task information

It is assumed you ran a Task in the previous section before running this section of the code. This allows you to see the output file ID to run further tasks on it, or simply download its contents. It also allows you to track the completion status of the task.

In [ ]:
taskID = json_format[0]["id"]
headers = {
    'Accept': 'application/json',
}
params = (
    ('id', taskID),
    ('cbrain_api_token', cbrain_api_token)
)
url = 'https://portal.cbrain.mcgill.ca/tasks/' + str(taskID)
response = requests.get(url, headers=headers, params=params)
if response.status_code == 200:
    print(response.text)
else:
    print("Task Info retrieval failed.")

## 4. Logout

In [ ]:
headers = {
    'Accept': 'application/json',
}
params = (
    ('cbrain_api_token', cbrain_api_token),
)
response = requests.delete('https://portal.cbrain.mcgill.ca/session', headers=headers, params=params)
if response.status_code == 200:
    print("Logout success")
else:
    print("Logout failure")